In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, when
import time

spark = SparkSession.builder.appName("HeavyTest").getOrCreate()

start = time.time()

# Generate 100k rows with IDs from 1 to 100000
df = spark.range(1, 100_001)

# Add synthetic features
df = df.withColumn("value", expr("id * 3 + 7"))
df = df.withColumn("category", when(col("id") % 2 == 0, "even").otherwise("odd"))
df = df.withColumn("flag", when(col("value") > 150_000, 1).otherwise(0))

# Aggregation
agg = df.groupBy("category", "flag").count()

# Show final result
agg.show()

print(f"Execution time: {time.time() - start:.2f} seconds")

spark.stop()


+--------+----+-----+
|category|flag|count|
+--------+----+-----+
|    even|   0|24998|
|     odd|   0|24999|
|    even|   1|25002|
|     odd|   1|25001|
+--------+----+-----+

Execution time: 1.75 seconds


In [2]:
import pandas as pd
import numpy as np
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum

# Create Spark session
spark = SparkSession.builder \
    .appName("Pandas_vs_PySpark_SpeedTest") \
    .getOrCreate()

# Set dataset size
num_rows = 10_000
num_groups = 1000

# Create sample data
print("Generating synthetic data...")
data = {
    'group': np.random.randint(0, num_groups, size=num_rows),
    'value': np.random.rand(num_rows)
}

# -----------------------------
# Pandas Benchmark
# -----------------------------
df_pandas = pd.DataFrame(data)

print("Running Pandas groupby...")
start_time = time.time()
pandas_result = df_pandas.groupby('group')['value'].sum()
pandas_time = time.time() - start_time
print(f"Pandas groupby took {pandas_time:.2f} seconds.")

# -----------------------------
# PySpark Benchmark
# -----------------------------
df_spark = spark.createDataFrame(df_pandas)

print("Running PySpark groupby...")
start_time = time.time()
spark_result = df_spark.groupBy("group").agg(spark_sum("value"))
# Force evaluation with collect()
spark_result.collect()
spark_time = time.time() - start_time
print(f"PySpark groupby took {spark_time:.2f} seconds.")

# -----------------------------
# Summary
# -----------------------------
print("\n=== Performance Summary ===")
print(f"Pandas:  {pandas_time:.2f} sec")
print(f"PySpark: {spark_time:.2f} sec")

# Stop Spark session
spark.stop()


Generating synthetic data...
Running Pandas groupby...
Pandas groupby took 0.19 seconds.
Running PySpark groupby...


Py4JJavaError: An error occurred while calling o98.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 8) (BlackPanther executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:252)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:143)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:158)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:178)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:261)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:70)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:374)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:338)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPart

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as spark_sum

# Start Spark session
spark = SparkSession.builder \
    .appName("BasicTest") \
    .master("local[*]") \
    .getOrCreate()

# Suppress logs
spark.sparkContext.setLogLevel("ERROR")

# Sample data (very small)
data = [(0, 10.0), (1, 20.0), (0, 5.0), (1, 15.0)]
columns = ["group", "value"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

# Run simple groupBy + sum
result = df.groupBy("group").agg(spark_sum("value"))

# Show result
result.show()

# Stop Spark session
spark.stop()


Py4JJavaError: An error occurred while calling o132.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 16) (BlackPanther executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:252)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:143)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:158)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:178)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:261)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:70)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:374)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:338)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartiti